## NDVI Computation with OpenEO

This notebook demonstrates computing NDVI from Sentinel-2 imagery using the titiler-OpenEO API.

In [ ]:
import requests
import base64
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from IPython.display import Image, display

## Configuration

In [ ]:
BASE_URL = "https://eoapi.develop.eoepca.org/openeo/"
STAC_URL = "https://eoapi.develop.eoepca.org/stac"

# Authenticate
auth_header = {"Authorization": f"Basic {base64.b64encode(b'test:test').decode('ascii')}"}
token = requests.get(f"{BASE_URL}credentials/basic", headers=auth_header).json()["access_token"]

## Get Scene Info from STAC

In [ ]:
item = requests.get(f"{STAC_URL}/collections/sentinel-2-ometepe/items/S2B_16PFT_20250412_0_L2A").json()
bbox = item["bbox"]
preview_url = next((item["assets"][k]["href"] for k in ["thumbnail", "preview", "visual"] if k in item["assets"]), item["assets"]["visual"]["href"])

## Display Preview

In [ ]:
display(Image(url=preview_url))

## Compute NDVI with OpenEO

In [ ]:
# Compute NDVI with proper Sentinel-2 scaling
# Steps: scale (0-10000 → 0-1), subtract offset (0.1), clamp to [0,1], then compute NDVI
process_graph = {
    "load1": {
        "process_id": "load_collection",
        "arguments": {
            "id": "sentinel-2-ometepe",
            "spatial_extent": {"west": bbox[0], "south": bbox[1], "east": bbox[2], "north": bbox[3]},
            "temporal_extent": ["2025-04-12T16:20:21Z", "2025-04-12T16:20:22Z"],
            "bands": ["red", "nir"]
        }
    },
    "red_band": {
        "process_id": "array_element",
        "arguments": {"data": {"from_node": "load1"}, "index": 0}
    },
    "red_scale": {
        "process_id": "linear_scale_range",
        "arguments": {
            "x": {"from_node": "red_band"},
            "inputMin": 0,
            "inputMax": 10000,
            "outputMin": 0.0,
            "outputMax": 1.0
        }
    },
    "red_reflectance": {
        "process_id": "subtract",
        "arguments": {"x": {"from_node": "red_scale"}, "y": 0.1}
    },
    "red_clamped": {
        "process_id": "clip",
        "arguments": {
            "x": {"from_node": "red_reflectance"},
            "min": 0.0,
            "max": 1.0
        }
    },
    "nir_band": {
        "process_id": "array_element",
        "arguments": {"data": {"from_node": "load1"}, "index": 1}
    },
    "nir_scale": {
        "process_id": "linear_scale_range",
        "arguments": {
            "x": {"from_node": "nir_band"},
            "inputMin": 0,
            "inputMax": 10000,
            "outputMin": 0.0,
            "outputMax": 1.0
        }
    },
    "nir_reflectance": {
        "process_id": "subtract",
        "arguments": {"x": {"from_node": "nir_scale"}, "y": 0.1}
    },
    "nir_clamped": {
        "process_id": "clip",
        "arguments": {
            "x": {"from_node": "nir_reflectance"},
            "min": 0.0,
            "max": 1.0
        }
    },
    "ndvi": {
        "process_id": "normalized_difference",
        "arguments": {
            "x": {"from_node": "nir_clamped"},
            "y": {"from_node": "red_clamped"}
        }
    },
    "save1": {
        "process_id": "save_result",
        "arguments": {"data": {"from_node": "ndvi"}, "format": "GTiff"},
        "result": True
    }
}

result = requests.post(
    f"{BASE_URL}result",
    json={"process": {"process_graph": process_graph}},
    headers={"Authorization": f"Bearer basic//{token}"}
)
result.raise_for_status()

with open("ndvi.tif", "wb") as f:
    f.write(result.content)
print(f"✓ NDVI computed via OpenEO: {len(result.content):,} bytes")

## Display NDVI Result

In [ ]:
with rasterio.open("ndvi.tif") as src:
    ndvi_data = src.read(1)
    valid_data = ndvi_data[~np.isnan(ndvi_data)]
    
    print(f"NDVI Statistics:")
    print(f"  Min: {valid_data.min():.3f}, Max: {valid_data.max():.3f}")
    print(f"  Mean: {valid_data.mean():.3f}, Median: {np.median(valid_data):.3f}")
    print(f"  Valid pixels: {len(valid_data):,}")
    
    plt.figure(figsize=(10, 10))
    im = plt.imshow(ndvi_data, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.title('NDVI Map')
    plt.axis('off')
    plt.colorbar(im, label='NDVI')
    plt.tight_layout()
    plt.show()